<a href="https://colab.research.google.com/github/TaibatNofiu/lending_club-accept/blob/main/Lending_Club_Data_Gathering_and_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import necessary libraries to perform

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Problem Statement
The dataset are information about the people that were given loan. We want to analyze the data to predict if a customer will default or not when given loan.

##1. Import Data
The dataset are in 2 seperate file, so we will import them and display them before merging together

In [2]:
accepted_data = pd.read_csv("/content/drive/MyDrive/GMC/accepted_2007_to_2018Q4.csv.gz", compression = 'gzip', low_memory= False)
accepted_data.shape

(2260701, 151)

### Display the columns of each data

In [5]:
accepted_data.columns.tolist()

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

##2. Data Cleaning


*   Reduce the columns size of accepted data by dropping columns with missing values higher than 60%
*   Use the zip_code column to add `region` column to the dataset
*   Add a new column (status) using the `debt_settlement_flag` column, this is to know the loans that were repaid and defaulted
*   Change the date type from object to datatime type





In [3]:
# Replace 'Unknown' values in 'issue_d' with NaT (Not a Time)
accepted_data['issue_d'] = accepted_data['issue_d'].replace('Unknown', pd.NaT)

# Convert 'issue_d' to datetime, handling errors gracefully
accepted_data['issue_d'] = pd.to_datetime(accepted_data['issue_d'], format= '%Y-%m-%d', errors='coerce')

In [4]:
## Add a column that use zip_code to define region
# Map first digit to region
region_map = {
    '0': 'Northeast',
    '1': 'Mid-Atlantic',
    '2': 'Mid-Atlantic & Southeast',
    '3': 'Southeast',
    '4': 'Midwest',
    '5': 'Upper Midwest',
    '6': 'Central',
    '7': 'South Central',
    '8': 'Mountain',
    '9': 'West / Pacific'
}

# Use map with the first character of zip_code
accepted_data['region'] = accepted_data['zip_code'].str[0].map(region_map)

In [5]:
len(accepted_data)

2260701

In [40]:
# Check for the missing values in descending order
missing_values = accepted_data.isna().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
missing_values

,0
member_id,2260701
issue_d,2260701
orig_projected_additional_accrued_interest,2252050
hardship_status,2249784
hardship_start_date,2249784
...,...
recoveries,33
total_rec_late_fee,33
debt_settlement_flag,33
disbursement_method,33


In [46]:
# Create a minimum and maximum threshold to drop the missing columns
low = 0.3 * len(accepted_data)
high = 2260701

columns_to_drop = missing_values[(missing_values >= low) & (missing_values <= high)].index
accepted_cleaned = accepted_data.drop(columns=columns_to_drop, axis = 1)
accepted_cleaned.shape

(2260701, 94)

In [47]:
accepted_cleaned.columns.tolist()

['id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'pymnt_plan',
 'url',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_with

In [49]:
accepted_cleaned['status'].value_counts()

,count
status,
1.0,1957056
0.0,303612


The loan status shows the people that repaid their loan and those that defaulted

We can a new column named status to show the people that repaid and those that defaulted using the loan_status column

In [21]:
# Create a new column using the debt_settlemet_flag Y is 1 and N 0, nan remains nan
loan_mapping = {
    'Fully Paid': 1,
    'Charged Off': 0,
    'Current': 1,
    'Late (31-120 days)': 0,
    'In Grace Period': 0,
    'Late (16-30 days)': 0,
    'Default': 0,
    'Does not meet the credit policy. Status:Fully Paid': 1,
    'Does not meet the credit policy. Status:Charged Off': 0,
    }

accepted_cleaned['status'] = accepted_cleaned['loan_status'].map(loan_mapping)
accepted_cleaned['status'].value_counts()

,count
status,
1.0,1957056
0.0,303612


In [22]:
# Employment length mapping
employment_length_mapping = {
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
    '< 1 year': 0.5,
    'n/a': 0
}

# Apply the mapping to the 'emp_length' column
accepted_cleaned['emp_length'] = accepted_cleaned['emp_length'].map(employment_length_mapping)
accepted_cleaned['emp_length'].value_counts()

,count
emp_length,
10.0,748005
2.0,203677
0.5,189988
3.0,180753
1.0,148403
5.0,139698
4.0,136605
6.0,102628
7.0,92695


##3. Perform Data Analysis Exploration

In [50]:
accepted_cleaned.shape

(2260701, 94)

In [51]:
accepted_cleaned.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,status
count,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.113761e+06,2.260664e+06,2.258957e+06,2.260639e+06,2.260668e+06,...,2.190392e+06,2.190237e+06,2.185289e+06,2.259303e+06,2.260563e+06,2.190392e+06,2.210638e+06,2.210638e+06,2.190392e+06,2.260668e+06
mean,1.504693e+04,1.504166e+04,1.502344e+04,1.309283e+01,4.458068e+02,5.976300e+00,7.799243e+04,1.882420e+01,3.068792e-01,6.985882e+02,...,2.076755e+00,9.411458e+01,4.243513e+01,1.281935e-01,4.677109e-02,1.782428e+05,5.102294e+04,2.319377e+04,4.373201e+04,8.656981e-01
std,9.190245e+03,9.188413e+03,9.192332e+03,4.832138e+00,2.671735e+02,3.650659e+00,1.126962e+05,1.418333e+01,8.672303e-01,3.301038e+01,...,1.830711e+00,9.036140e+00,3.621616e+01,3.646130e-01,3.775338e-01,1.815748e+05,4.991124e+04,2.300656e+04,4.507298e+04,3.409765e-01
min,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,5.000000e-01,0.000000e+00,-1.000000e+00,0.000000e+00,6.100000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,2.000000e+00,4.600000e+04,1.189000e+01,0.000000e+00,6.750000e+02,...,1.000000e+00,9.130000e+01,0.000000e+00,0.000000e+00,0.000000e+00,5.073100e+04,2.089200e+04,8.300000e+03,1.500000e+04,1.000000e+00
50%,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.000000e+00,6.500000e+04,1.784000e+01,0.000000e+00,6.900000e+02,...,2.000000e+00,1.000000e+02,3.750000e+01,0.000000e+00,0.000000e+00,1.142985e+05,3.786400e+04,1.630000e+04,3.269600e+04,1.000000e+00
75%,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,1.000000e+01,9.300000e+04,2.449000e+01,0.000000e+00,7.150000e+02,...,3.000000e+00,1.000000e+02,7.140000e+01,0.000000e+00,0.000000e+00,2.577550e+05,6.435000e+04,3.030000e+04,5.880425e+04,1.000000e+00
max,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.000000e+01,1.100000e+08,9.990000e+02,5.800000e+01,8.450000e+02,...,3.200000e+01,1.000000e+02,1.000000e+02,1.200000e+01,8.500000e+01,9.999999e+06,3.408095e+06,1.569000e+06,2.118996e+06,1.000000e+00


In [59]:
accepted_cleaned.describe(include = 'object')

,id,term,grade,sub_grade,emp_title,home_ownership,verification_status,loan_status,pymnt_plan,url,...,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag,region
count,2260701,2260668,2260668,2260668,2093699,2260668,2260668,2260668,2260668,2260668,...,2260668,2260639,2260668,2258241,2260596,2260668,2260668,2260668,2260668,2260667
unique,2260701,2,7,35,512694,6,3,9,2,2260668,...,51,754,2,136,141,2,2,2,2,10
top,Total amount funded in policy code 2: 521953170,36 months,B,C1,Teacher,MORTGAGE,Source Verified,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,...,CA,Sep-2004,w,Mar-2019,Mar-2019,Individual,N,Cash,N,West / Pacific
freq,1,1609754,663557,145903,38824,1111450,886231,1076751,2260048,1,...,314533,15400,1535467,853003,1371381,2139958,2259836,2182546,2226422,404303


In [52]:
accepted_cleaned.shape

(2260701, 94)

In [53]:
accepted_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 94 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   id                          object 
 1   loan_amnt                   float64
 2   funded_amnt                 float64
 3   funded_amnt_inv             float64
 4   term                        object 
 5   int_rate                    float64
 6   installment                 float64
 7   grade                       object 
 8   sub_grade                   object 
 9   emp_title                   object 
 10  emp_length                  float64
 11  home_ownership              object 
 12  annual_inc                  float64
 13  verification_status         object 
 14  loan_status                 object 
 15  pymnt_plan                  object 
 16  url                         object 
 17  purpose                     object 
 18  title                       object 
 19  zip_code             

In [54]:
category_columns = accepted_cleaned.select_dtypes(include = ['object']).columns
category_columns

Index(['id', 'term', 'grade', 'sub_grade', 'emp_title', 'home_ownership',
       'verification_status', 'loan_status', 'pymnt_plan', 'url', 'purpose',
       'title', 'zip_code', 'addr_state', 'earliest_cr_line',
       'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d',
       'application_type', 'hardship_flag', 'disbursement_method',
       'debt_settlement_flag', 'region'],
      dtype='object')

In [55]:
numerical_columns = accepted_cleaned.select_dtypes(include = ['float64', 'int64']).columns
numerical_columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high',
       'last_fico_range_low', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev

In [56]:
accepted_cleaned.duplicated().sum()

np.int64(0)

In [57]:
accepted_cleaned.nunique().sort_values()

,0
policy_code,1
term,2
initial_list_status,2
pymnt_plan,2
application_type,2
...,...
last_pymnt_amnt,704467
total_pymnt_inv,1311099
total_pymnt,1633864
url,2260668


In [58]:
accepted_cleaned['status'].value_counts(normalize = True)

,proportion
status,
1.0,0.865698
0.0,0.134302


In [60]:
accepted_cleaned[numerical_columns].describe().T[['min', 'max']]

,min,max
loan_amnt,500.00,40000.00
funded_amnt,500.00,40000.00
funded_amnt_inv,0.00,40000.00
int_rate,5.31,30.99
installment,4.93,1719.83
...,...,...
tot_hi_cred_lim,0.00,9999999.00
total_bal_ex_mort,0.00,3408095.00
total_bc_limit,0.00,1569000.00
total_il_high_credit_limit,0.00,2118996.00
